In [1]:
import pandas as pd
from math import *
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np
import time, threading

In [2]:
pd.set_option('display.max_columns', None)
#read in data from cosmic using path to file
df = pd.read_csv(r"C:\Users\bethc\downloads\COSMICfilteredPop100KWDWDcrossover.csv")
lisasens=pd.read_csv(r"C:\Users\bethc\Downloads\LISAcurveSigFig.csv", index_col=False)

print(df.columns.values)

['Unnamed: 0' 'PorbSI' 'mass_1SI' 'mass_2SI' 'mass_1rSI' 'mass_2rSI' 'mc'
 'ecc' 'a_b' 'w_spin1' 'w_spin2' 'wl' 'wh' 'w_force' 'w_star1' 'w_star2'
 'kstar_1' 'kstar_2']


In [3]:
w_force=df.w_force
wl=df.wl
wh=df.wh

In [4]:
cond = (df['w_force'] < df['wl'] ) & (df['w_force'].shift(1) >= df.wl)



In [5]:
#histogram of wforce/wforce_max and wforce/wforce_min
#wforce_max when star is last state
#wforce_min when star is first CO WD

In [6]:
stip=w_force/wl-w_force/wh

In [7]:
stip_true=stip<0
stip_false=stip>0

In [8]:
stip_true

0         True
1         True
2         True
3         True
4         True
          ... 
707483    True
707484    True
707485    True
707486    True
707487    True
Length: 707488, dtype: bool

In [9]:
#scatter plot: LISA SNR vs. stip_true(discriminator)
#SNR<1,2,10 w/10 being desirable
#wforce/wl vs. wforce/wh

In [10]:
flisa=lisasens.f
hflisa=lisasens.hf
#constants

G=6.67408*10**-11
c=3.0e8
#convert orbital period from days to seconds
porbSI=df.PorbSI
#convert mass1/2 from solar mass to kg
mass_1SI=df.mass_1SI
mass_2SI=df.mass_2SI
#calculate components for SNR
#frequency
f=2/porbSI
# f=f.replace([np.inf,-np.inf, [0]],np.nan)
# f.dropna(inplace=True)
#chirp mass
mc=df.mc
# mc=mc.replace([0],np.nan)
# mc.dropna(inplace=True)
#distance between binaries
dist=((G*mc*porbSI**2)/(4*pi**2))**(1/3)
#distance from binary system to LISA
#assume distance of 1200pc
#convert to meters
dist_LISA=1200*3.0857e16
#strain
h_0=4*(G/c**2)*(mc/dist_LISA)*((G/c**3)*pi*f*mc)**(2/3)
h_0=h_0.replace([np.inf,-np.inf],np.nan)
h_0.dropna(inplace=True)
#observing time in years
#converting to seconds
tobs=1.262e8
#monochromatic source
h_f_mono=h_0*sqrt(tobs)
#LISA sensitivity
#sort/log LISA data
log_flisa=np.log(flisa)
log_flisa_res=[]
for i in log_flisa:
    if i not in log_flisa_res:
        log_flisa_res.append(i)
sort_flisa=sorted(log_flisa_res)
log_hflisa=np.log(hflisa)
log_hflisa_res=[]
for i in log_hflisa:
    if i not in log_hflisa_res:
        log_hflisa_res.append(i)
sort_hflisa=sorted(log_hflisa_res)
#create spline fit of LISA curve
s=interpolate.InterpolatedUnivariateSpline(flisa,hflisa)
h_f_d=s(h_f_mono)*sqrt(tobs)
#for chirping sources
kappa=5/256*((G/c**3)*mc)**(-5/3)*pi**(-8/3)
#stationary freq
fs=(8/3*kappa/(tobs**2))**(3/11)
fs
#tbin
tbin=8/3*kappa*fs**(-11/3)/tobs

In [11]:
#SNR=np.where(f<=fs,np.log(h_f_mono)/np.log(h_f_d),(((h_0**2)/(h_f_d**2))*tbin)**(1/2))

for i in range(len(f)):
    if f[i]<=fs[i]:
        SNR=np.log(h_f_mono)/np.log(h_f_d)
        
    elif f[i]>fs[i]:
        SNR=(((h_0**2)/(h_f_d**2))*tbin)**(1/2)
        

KeyboardInterrupt: 

In [12]:
len(SNR)

707465

In [ ]:
len(h_f_d)

In [ ]:
len(h_0)

In [ ]:
len(f)

In [ ]:
len(fs)

In [ ]:
range(len(f))

In [ ]:
# #create plot of lisa curve with SNR scatter
# plt.xscale('log')
# plt.yscale('log')
# plt.scatter(flisa,hflisa)
# plt.scatter(f,stip_true)
# plt.show()

In [ ]:
plt.scatter(w_force/wl, w_force/wh)
plt.show()

In [ ]:
max(w_force/wl)

In [ ]:
max(w_force/wh)

In [ ]:
min(w_force/wl)

In [ ]:
min(w_force/wh)